In [130]:
import numpy as np
import pandas as pd

from us import states

from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

from sqlalchemy import create_engine
import pandas.io.sql as pd_sql

In [83]:
results_2012 = pd.read_csv('data/2012_results.csv')
results_2014 = pd.read_csv('data/2014_results.csv')
results_2016 = pd.read_csv('data/2016_results.csv')
results_2018 = pd.read_csv('data/2018_results.csv')

In [84]:
dfs = [results_2012,results_2014,results_2016,results_2018]

In [85]:
results_2012.head()

,Name,URL,State,District,Party,Terms
0,"Representative Adams, Alma S.",https://www.congress.gov/member/alma-adams/A00...,North Carolina,12.0,Democratic,House: 2014-Present
1,"Representative Aderholt, Robert B.",https://www.congress.gov/member/robert-aderhol...,Alabama,4.0,Republican,House: 1997-Present
2,"Representative Alexander, Rodney",https://www.congress.gov/member/rodney-alexand...,Louisiana,5.0,Republican,House: 2003-2013
3,"Representative Amash, Justin",https://www.congress.gov/member/justin-amash/A...,Michigan,3.0,Republican,House: 2011-Present
4,"Representative Amodei, Mark E.",https://www.congress.gov/member/mark-amodei/A0...,Nevada,2.0,Republican,House: 2011-Present


In [86]:
st = states.STATES

In [87]:
states_dict = {}

for state in st:
    states_dict[state] = state.fips

In [88]:
fips = {}
for state, fip in states_dict.items():
    fips[str(state)] = fip

In [89]:
fips

{'Alabama': '01',
 'Alaska': '02',
 'Arizona': '04',
 'Arkansas': '05',
 'California': '06',
 'Colorado': '08',
 'Connecticut': '09',
 'Delaware': '10',
 'District of Columbia': '11',
 'Florida': '12',
 'Georgia': '13',
 'Hawaii': '15',
 'Idaho': '16',
 'Illinois': '17',
 'Indiana': '18',
 'Iowa': '19',
 'Kansas': '20',
 'Kentucky': '21',
 'Louisiana': '22',
 'Maine': '23',
 'Maryland': '24',
 'Massachusetts': '25',
 'Michigan': '26',
 'Minnesota': '27',
 'Mississippi': '28',
 'Missouri': '29',
 'Montana': '30',
 'Nebraska': '31',
 'Nevada': '32',
 'New Hampshire': '33',
 'New Jersey': '34',
 'New Mexico': '35',
 'New York': '36',
 'North Carolina': '37',
 'North Dakota': '38',
 'Ohio': '39',
 'Oklahoma': '40',
 'Oregon': '41',
 'Pennsylvania': '42',
 'Rhode Island': '44',
 'South Carolina': '45',
 'South Dakota': '46',
 'Tennessee': '47',
 'Texas': '48',
 'Utah': '49',
 'Vermont': '50',
 'Virginia': '51',
 'Washington': '53',
 'West Virginia': '54',
 'Wisconsin': '55',
 'Wyoming': '56

In [90]:
states_list = list(fips.keys())

In [91]:
drop_list

['Guam',
 'Virgin Islands',
 'American Samoa',
 'Puerto Rico',
 'Northern Mariana Islands',
 'Guam',
 'Puerto Rico',
 'Virgin Islands',
 'American Samoa',
 'Northern Mariana Islands',
 'Guam',
 'Puerto Rico',
 'Virgin Islands',
 'American Samoa',
 'Northern Mariana Islands',
 'Puerto Rico',
 'Virgin Islands',
 'American Samoa',
 'Northern Mariana Islands',
 'Guam']

In [93]:
for df in dfs:
    print(len(df))

450
447
456
440


In [97]:
drop_indices = []
for index, row in dfs[3]['State'].iteritems():
    if row in drop_set:
        drop_indices.append(index)
dfs[3].drop(drop_indices, inplace=True)

In [98]:
for df in dfs:
    print(len(df))

445
442
451
435


In [69]:
drop_indices

[29, 64, 124, 317, 360]

In [99]:
for frame in dfs:
    frame['FIPS'] = [fips[fi] for fi in frame['State']]

In [100]:
for frame in dfs:
    print(len(frame))

445
442
451
435


In [104]:
results_2018.head(10)

,Name,URL,State,District,Party,Terms,FIPS
0,"Representative Abraham, Ralph Lee",https://www.congress.gov/member/ralph-abraham/...,Louisiana,5.0,Republican,House: 2015-Present,22
1,"Representative Adams, Alma S.",https://www.congress.gov/member/alma-adams/A00...,North Carolina,12.0,Democratic,House: 2014-Present,37
2,"Representative Aderholt, Robert B.",https://www.congress.gov/member/robert-aderhol...,Alabama,4.0,Republican,House: 1997-Present,01
3,"Representative Aguilar, Pete",https://www.congress.gov/member/pete-aguilar/A...,California,31.0,Democratic,House: 2015-Present,06
4,"Representative Allen, Rick W.",https://www.congress.gov/member/rick-allen/A00...,Georgia,12.0,Republican,House: 2015-Present,13
5,"Representative Allred, Colin Z.",https://www.congress.gov/member/colin-allred/A...,Texas,32.0,Democratic,House: 2019-Present,48
6,"Representative Amash, Justin",https://www.congress.gov/member/justin-amash/A...,Michigan,3.0,Republican,House: 2011-Present,26
7,"Representative Amodei, Mark E.",https://www.congress.gov/member/mark-amodei/A0...,Nevada,2.0,Republican,House: 2011-Present,32
8,"Representative Armstrong, Kelly",https://www.congress.gov/member/kelly-armstron...,North Dakota,NaN,Republican,House: 2019-Present,38
9,"Representative Arrington, Jodey C.",https://www.congress.gov/member/jodey-arringto...,Texas,19.0,Republican,House: 2017-Present,48


In [109]:
results_2012.fillna(value=0,inplace=True)

In [110]:
results_2014.fillna(value=0,inplace=True)
results_2016.fillna(value=0,inplace=True)
results_2018.fillna(value=0,inplace=True)

In [113]:
results_2012 = results_2012.rename(columns={'State': 'Oldstate', 'FIPS': 'State'})

In [116]:
results_2014 = results_2014.rename(columns={'State': 'Oldstate', 'FIPS': 'State'})
results_2016 = results_2016.rename(columns={'State': 'Oldstate', 'FIPS': 'State'})
results_2018 = results_2018.rename(columns={'State': 'Oldstate', 'FIPS': 'State'})

In [117]:
results_2018.head(10)

,Name,URL,Oldstate,District,Party,Terms,State
0,"Representative Abraham, Ralph Lee",https://www.congress.gov/member/ralph-abraham/...,Louisiana,5.0,Republican,House: 2015-Present,22
1,"Representative Adams, Alma S.",https://www.congress.gov/member/alma-adams/A00...,North Carolina,12.0,Democratic,House: 2014-Present,37
2,"Representative Aderholt, Robert B.",https://www.congress.gov/member/robert-aderhol...,Alabama,4.0,Republican,House: 1997-Present,01
3,"Representative Aguilar, Pete",https://www.congress.gov/member/pete-aguilar/A...,California,31.0,Democratic,House: 2015-Present,06
4,"Representative Allen, Rick W.",https://www.congress.gov/member/rick-allen/A00...,Georgia,12.0,Republican,House: 2015-Present,13
5,"Representative Allred, Colin Z.",https://www.congress.gov/member/colin-allred/A...,Texas,32.0,Democratic,House: 2019-Present,48
6,"Representative Amash, Justin",https://www.congress.gov/member/justin-amash/A...,Michigan,3.0,Republican,House: 2011-Present,26
7,"Representative Amodei, Mark E.",https://www.congress.gov/member/mark-amodei/A0...,Nevada,2.0,Republican,House: 2011-Present,32
8,"Representative Armstrong, Kelly",https://www.congress.gov/member/kelly-armstron...,North Dakota,0.0,Republican,House: 2019-Present,38
9,"Representative Arrington, Jodey C.",https://www.congress.gov/member/jodey-arringto...,Texas,19.0,Republican,House: 2017-Present,48


In [118]:
dfs = [results_2012,results_2014,results_2016,results_2018]

In [119]:
for df in dfs:
    df['District'] = df['District'].astype(int)
    df['State'] = df['State'].astype(int)

In [121]:
for df in dfs:
    df.reset_index(inplace=True)

In [123]:
results_2012['Year'] = 2012
results_2014['Year'] = 2014
results_2016['Year'] = 2016
results_2018['Year'] = 2018

In [128]:
results_2012.drop(['index'], axis=1, inplace=True)
results_2014.drop(['index'], axis=1, inplace=True)
results_2016.drop(['index'], axis=1, inplace=True)
results_2018.drop(['index'], axis=1, inplace=True)

In [132]:
connection_string = f'postgresql://ubuntu@54.152.93.181:5432/elections'

engine = create_engine(connection_string)

In [134]:
results_2012.to_sql('results', engine, index=False,if_exists='append')

In [135]:
results_2014.to_sql('results', engine, index=False,if_exists='append')
results_2016.to_sql('results', engine, index=False,if_exists='append')
results_2018.to_sql('results', engine, index=False,if_exists='append')

In [140]:
results_data = results_2012.append(results_2014)
results_data = results_data.append(results_2016)
results_data = results_data.append(results_2018)

In [141]:
len(results_data)

1773

In [142]:
results_data.to_csv('results_data.csv')

In [131]:
results_2016

,Name,URL,Oldstate,District,Party,Terms,State,Year
0,"Representative Abraham, Ralph Lee",https://www.congress.gov/member/ralph-abraham/...,Louisiana,5,Republican,House: 2015-Present,22,2016
1,"Representative Adams, Alma S.",https://www.congress.gov/member/alma-adams/A00...,North Carolina,12,Democratic,House: 2014-Present,37,2016
2,"Representative Aderholt, Robert B.",https://www.congress.gov/member/robert-aderhol...,Alabama,4,Republican,House: 1997-Present,1,2016
3,"Representative Aguilar, Pete",https://www.congress.gov/member/pete-aguilar/A...,California,31,Democratic,House: 2015-Present,6,2016
4,"Representative Allen, Rick W.",https://www.congress.gov/member/rick-allen/A00...,Georgia,12,Republican,House: 2015-Present,13,2016
5,"Representative Amash, Justin",https://www.congress.gov/member/justin-amash/A...,Michigan,3,Republican,House: 2011-Present,26,2016
6,"Representative Amodei, Mark E.",https://www.congress.gov/member/mark-amodei/A0...,Nevada,2,Republican,House: 2011-Present,32,2016
7,"Representative Arrington, Jodey C.",https://www.congress.gov/member/jodey-arringto...,Texas,19,Republican,House: 2017-Present,48,2016
8,"Representative Babin, Brian",https://www.congress.gov/member/brian-babin/B0...,Texas,36,Republican,House: 2015-Present,48,2016
9,"Representative Bacon, Don",https://www.congress.gov/member/don-bacon/B001298,Nebraska,2,Republican,House: 2017-Present,31,2016


In [101]:
results_2012.head()

,Name,URL,State,District,Party,Terms,FIPS
0,"Representative Adams, Alma S.",https://www.congress.gov/member/alma-adams/A00...,North Carolina,12.0,Democratic,House: 2014-Present,37
1,"Representative Aderholt, Robert B.",https://www.congress.gov/member/robert-aderhol...,Alabama,4.0,Republican,House: 1997-Present,01
2,"Representative Alexander, Rodney",https://www.congress.gov/member/rodney-alexand...,Louisiana,5.0,Republican,House: 2003-2013,22
3,"Representative Amash, Justin",https://www.congress.gov/member/justin-amash/A...,Michigan,3.0,Republican,House: 2011-Present,26
4,"Representative Amodei, Mark E.",https://www.congress.gov/member/mark-amodei/A0...,Nevada,2.0,Republican,House: 2011-Present,32


In [32]:
states_list = []
for state in st:
    str_st = str(state)
    if str_st not in states_list:
        states_list.append(str_st)

In [33]:
states_list

['Alabama',
 'Alaska',
 'Arizona',
 'Arkansas',
 'California',
 'Colorado',
 'Connecticut',
 'Delaware',
 'District of Columbia',
 'Florida',
 'Georgia',
 'Hawaii',
 'Idaho',
 'Illinois',
 'Indiana',
 'Iowa',
 'Kansas',
 'Kentucky',
 'Louisiana',
 'Maine',
 'Maryland',
 'Massachusetts',
 'Michigan',
 'Minnesota',
 'Mississippi',
 'Missouri',
 'Montana',
 'Nebraska',
 'Nevada',
 'New Hampshire',
 'New Jersey',
 'New Mexico',
 'New York',
 'North Carolina',
 'North Dakota',
 'Ohio',
 'Oklahoma',
 'Oregon',
 'Pennsylvania',
 'Rhode Island',
 'South Carolina',
 'South Dakota',
 'Tennessee',
 'Texas',
 'Utah',
 'Vermont',
 'Virginia',
 'Washington',
 'West Virginia',
 'Wisconsin',
 'Wyoming']

In [17]:
fips = {}
for state in st:
    fips[state] = state.fips

In [18]:
fips

{<State:Alabama>: '01',
 <State:Alaska>: '02',
 <State:Arizona>: '04',
 <State:Arkansas>: '05',
 <State:California>: '06',
 <State:Colorado>: '08',
 <State:Connecticut>: '09',
 <State:Delaware>: '10',
 <State:District of Columbia>: '11',
 <State:Florida>: '12',
 <State:Georgia>: '13',
 <State:Hawaii>: '15',
 <State:Idaho>: '16',
 <State:Illinois>: '17',
 <State:Indiana>: '18',
 <State:Iowa>: '19',
 <State:Kansas>: '20',
 <State:Kentucky>: '21',
 <State:Louisiana>: '22',
 <State:Maine>: '23',
 <State:Maryland>: '24',
 <State:Massachusetts>: '25',
 <State:Michigan>: '26',
 <State:Minnesota>: '27',
 <State:Mississippi>: '28',
 <State:Missouri>: '29',
 <State:Montana>: '30',
 <State:Nebraska>: '31',
 <State:Nevada>: '32',
 <State:New Hampshire>: '33',
 <State:New Jersey>: '34',
 <State:New Mexico>: '35',
 <State:New York>: '36',
 <State:North Carolina>: '37',
 <State:North Dakota>: '38',
 <State:Ohio>: '39',
 <State:Oklahoma>: '40',
 <State:Oregon>: '41',
 <State:Pennsylvania>: '42',
 <Sta